In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import json


data_root = '../../MyData/'

In [46]:
desc = pd.read_csv(data_root+'merged_1197.csv')
desc

,cik,Year,item1,item1_word_count,item1_token_count,item1_summary,Error,ErrorMessage,item1_summary_token_count,item1_summary_word_count,...,SP_CUSIP,SP_TICKER,SP_EXCHANGE_TICKER,SP_COMPANY_NAME,_merge,word_count,YH_BUS_DESC,SA_BUS_DESC,ORBIS_PROD_SERV,ORBIS_OVERVIEW
0,850460,2021,PART I Item 1. Business. General Encor...,2816,3495,**Summary of Encore Wire Corporation's Busines...,0.0,NaN,737,549,...,292562105,WIRE,NASDAQGM:WIRE,Encore Wire Corporation,both,NaN,NaN,NaN,NaN,NaN
1,352541,2021,ITEM 1. BUSINESS A. GENERAL Alliant Energy...,7220,10062,### Summary of Alliant Energy's Business (Item...,0.0,NaN,596,463,...,018802108,LNT,NASDAQGS:LNT,Alliant Energy Corporation,both,NaN,Alliant Energy Corporation operates as a utili...,Alliant Energy Corporation operates as a utili...,Regulated electricity and natural gas services,"This company, formerly known as Interstate Ene..."
2,1704715,2021,Item 1. Business Unless otherwise indicated or...,3422,4414,**Summary of Item 1: Business**\n\n**Core Busi...,0.0,NaN,575,416,...,020764106,AMR,NYSE:AMR,"Alpha Metallurgical Resources, Inc.",both,NaN,"Alpha Metallurgical Resources, Inc., a mining ...","Alpha Metallurgical Resources, Inc., a mining ...",Mining operations across multiple major coal b...,"This company, based in the United States of Am..."
3,1575515,2021,Item 1. Business Sprouts Farmers Market offers...,6852,8149,**Summary of Sprouts Farmers Market Business O...,0.0,NaN,625,469,...,85208M102,SFM,NASDAQGS:SFM,"Sprouts Farmers Market, Inc.",both,NaN,"Sprouts Farmers Market, Inc., together with it...","Sprouts Farmers Market, Inc., together with it...",Natural and organic foods,This company is engaged in the retail sale of ...
4,1125376,2021,"Item 1. BUSINESS Founded in 1999, The Ensign G...",13563,16910,"**Summary of The Ensign Group, Inc. Business O...",0.0,NaN,644,480,...,29358P101,ENSG,NASDAQGS:ENSG,"The Ensign Group, Inc.",both,NaN,"The Ensign Group, Inc. provides skilled nursin...","The Ensign Group, Inc. provides skilled nursin...",Nursing and rehabilitative care services,"This company, based in the United States, oper..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,1021860,2021,ITEM 1. BUSINESS General NOV Inc. (NOV or the ...,9840,11886,**Summary of NOV Inc. Business Overview from 1...,0.0,NaN,582,453,...,62955J103,NOV,NYSE:NOV,NOV Inc.,both,NaN,"NOV Inc. designs, constructs, manufactures, an...","NOV Inc. designs, constructs, manufactures, an...","Systems, components, and products to the oil a...","This company is engaged in the design, manufac..."
1193,1744489,2021,PART I ITEM 1. Business The Walt Disney ...,11263,14519,### Summary of The Walt Disney Company's Busin...,0.0,NaN,717,512,...,254687106,DIS,NYSE:DIS,The Walt Disney Company,both,NaN,The Walt Disney Company operates as an enterta...,The Walt Disney Company operates as an enterta...,Motion picture and video production,This company is engaged in motion picture and ...
1194,1102112,2021,"ITEM 1. BUSINESS General PacWest Bancorp, a De...",12822,15799,**Summary of PacWest Bancorp's Business Overvi...,0.0,NaN,519,379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1195,28823,2021,ITEM 16: FORM 10-K SUMMARY 107 SIGNATURE...,3763,4762,"**Summary of Diebold Nixdorf, Incorporated's B...",0.0,NaN,611,428,...,NaN,NaN,NaN,NaN,NaN,NaN,"Diebold Nixdorf, Incorporated engages in the a...","Diebold Nixdorf, Incorporated engages in the a...",NaN,NaN


In [48]:
desc.columns

Index(['cik', 'Year', 'item1', 'item1_word_count', 'item1_token_count',
       'item1_summary', 'Error', 'ErrorMessage', 'item1_summary_token_count',
       'item1_summary_word_count', 'item1_embeddings', 'tic', 'Unnamed: 0',
       'conm', 'gvkey', 'SIC_Division', 'SIC_MajorGroup', 'NAICS_Sector',
       'NAICS_SubSector', 'hoberg_fic25', 'hoberg_fic100', 'GICS_Sector',
       'GICS_IndustryGroup', 'GICS_Industry', 'cusip', 'SP_ENTITY_NAME',
       'SP_ENTITY_ID', 'SP_SHORT_DESC', 'SP_LONG_DESC', 'SP_TOPICTAG',
       'SP_CATEGORIES', 'SP_CUSIP', 'SP_TICKER', 'SP_EXCHANGE_TICKER',
       'SP_COMPANY_NAME', '_merge', 'word_count', 'YH_BUS_DESC', 'SA_BUS_DESC',
       'ORBIS_PROD_SERV', 'ORBIS_OVERVIEW'],
      dtype='object')

In [33]:
desc['SP_SHORT_DESC']

0       Encore Wire Corporation engages in manufacture...
1       Alliant Energy is the parent company of two pu...
2       Alpha Metallurgical Resources, Inc., a mining ...
3       Sprouts Farmers Market, Inc., together with it...
4       The Ensign Group, Inc. provides skilled nursin...
                              ...                        
1192    NOV Inc. designs, constructs, manufactures, an...
1193    The Walt Disney Company operates as an enterta...
1194                                                  NaN
1195                                                  NaN
1196    United Airlines Holdings, Inc., through its su...
Name: SP_SHORT_DESC, Length: 1197, dtype: object

In [5]:
desc['SA_BUS_DESC'] = desc['SA_BUS_DESC'].str.replace(r"More\.\.\.$", "", regex=True)

In [40]:
checkres = desc['SP_SHORT_DESC'] == desc['SP_LONG_DESC']
checkres[checkres == True]

142    True
dtype: bool

In [12]:
desc['ORBIS_PROD_SERV'].isna().sum()


200

In [ ]:
import difflib

# Example strings
string1 = desc['SP_LONG_BUSINESS_DESCRIPTION'][142]
string2 = desc['SP_BUSINESS_DESCRIPTION'][142]

# Find differences
diff = difflib.ndiff(string1, string2)
print("\n".join(diff))


In [35]:
print(desc['ORBIS_PROD_SERV'].isna().sum())

count = desc['ORBIS_PROD_SERV'].str.contains(r'\[.*\]$', regex=True).sum()
count

200


997

In [43]:
desc['ORBIS_PROD_SERV'] = desc['ORBIS_PROD_SERV'].str.replace(r'\[.*?\]', '', regex=True)

In [47]:
desc['ORBIS_PROD_SERV'][1]

'Regulated electricity and natural gas services '

In [45]:
# desc.to_csv(data_root+"merged_1197.csv", index=False)